# selenium-yahoo-finance-scraper

Use the "Run" button to execute the code.

In [1]:
!pip install jovian --upgrade --quiet
!pip install selenium --upgrade --quiet

##prereq

- !pip install webdriver-manager --upgrade --quiet
- download required chromdriver and place it in the project path 

![](https://imgur.com/caHD7Dm.png)

![](https://imgur.com/H2LfYlh.png)


![](https://imgur.com/7jMFOcE.png)


![](https://media.giphy.com/media/TwLrWWtF6zcDGk5BQf/giphy.gif)

In [2]:
import jovian

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="selenium-yahoo-finance-scraper")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/selenium-yahoo-finance-scraper" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper


'https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper'

In [4]:
!pip install webdriver-manager --upgrade --quiet

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [6]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')
    serv = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=serv)
    driver = webdriver.Chrome(options=chrome_options, service=serv)
    return driver

In [7]:
def get_tickers(driver):
    TABLE_CLASS = "W(100%)"  
    driver.get(YAHOO_FINANCE_URL)
    tablerows = len(driver.find_elements(By.XPATH, value="//table[@class= '{}']/tbody/tr".format(TABLE_CLASS)))
    return tablerows

In [8]:
def parse_ticker(rownum, table_driver):
    Symbol = table_driver.find_element(By.XPATH, value="//tr[{}]/td[1]".format(rownum)).text
    Name = table_driver.find_element(By.XPATH, value="//tr[{}]/td[2]".format(rownum)).text
    LastPrice = table_driver.find_element(By.XPATH, value="//tr[{}]/td[3]".format(rownum)).text
    MarketTime = table_driver.find_element(By.XPATH, value="//tr[{}]/td[4]".format(rownum)).text
    Change = table_driver.find_element(By.XPATH, value="//tr[{}]/td[5]".format(rownum)).text
    PercentChange = table_driver.find_element(By.XPATH, value="//tr[{}]/td[6]".format(rownum)).text	
    Volume = table_driver.find_element(By.XPATH, value="//tr[{}]/td[7]".format(rownum)).text
    MarketCap = table_driver.find_element(By.XPATH, value="//tr[{}]/td[8]".format(rownum)).text	

    return {
    'Symbol': Symbol,
    'Name': Name,
    'LastPrice': LastPrice,
    'MarketTime': MarketTime,
    'Change': Change,
    'PercentChange': PercentChange,
    'Volume': Volume,
    'MarketCap': MarketCap
    }

In [12]:
YAHOO_FINANCE_URL = 'https://finance.yahoo.com/trending-tickers'

print('Creating driver')
driver = get_driver()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


Creating driver


Driver [/Users/vinoddhole/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [13]:
get_tickers(driver)

29

In [17]:
header = driver.find_elements(By.TAG_NAME, value= 'th')

In [22]:
header[0].text

'Symbol'

In [28]:
rownum=2
txt=driver.find_element(By.XPATH, value="//tr[{}]/td[2]".format(rownum)).text
txt

'MOEX Russia Index'

In [13]:
YAHOO_FINANCE_URL = 'https://finance.yahoo.com/trending-tickers'

print('Creating driver')
driver = get_driver()

print('Fetching the page')
table_rows = get_tickers(driver)

print(f'Found {table_rows} Tickers')

print('Parsing Trending tickers')
ticker_data = [parse_ticker(i, driver) for i in range (1, table_rows + 1)]

print('Save the data to a CSV')
videos_df = pd.DataFrame(ticker_data)
#print(videos_df)
videos_df.to_csv('trending-tickers.csv', index=None)




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


Creating driver


Driver [/Users/vinoddhole/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


Fetching the page
Found 30 Tickers
Parsing Trending tickers
Save the data to a CSV


In [1]:
jovian.commit(project="selenium-yahoo-finance-scraper",git_commit=True)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/selenium-yahoo-finance-scraper" on https://jovian.ai
[jovian] Git repository identified. Performing git commit...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'jovyan@jupyter-vinodvidhole--api-2dgit-2da27f5c9-2d4363c6a3b607-5f26-2.(none)')


'https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper'